In [13]:
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.datasets import CIFAR10,ImageFolder
from torch.utils.data import DataLoader
import torch
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from tqdm import tqdm

In [2]:
#Load Dataset


#set the transformation

transformation = transforms.Compose([
    
    transforms.Resize(64),
    transforms.ToTensor(),
  
])

dataset = CIFAR10(root='data/', download=True, transform=transformation)
test_dataset = CIFAR10(root='data/', train=False, transform=transformation)

Files already downloaded and verified


In [3]:
classes = dataset.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
torch.manual_seed(43)  #Reason behind setting random seed is that when you guys run this you will get the same split
val_size = 5000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(45000, 5000)

In [5]:
batch_size = 32

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

In [6]:
for images, _ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))

    break

KeyboardInterrupt: 

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
class Cifar10_Model(nn.Module):
    def __init__(self,number_of_classes):
        super().__init__()
        
        self.convo_Block_1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=(2,2))
            
        )
        
        self.convo_Block_2 = nn.Sequential(
            
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),stride=1,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=(2,2))
            
        )
        
        self.linear_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64*16*16,out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024,out_features=10)
        )
        
    def forward(self,x):
        x = self.convo_Block_1(x)
        x = self.convo_Block_2(x)
        x = self.linear_layer(x)
        
        return x

In [9]:
model = Cifar10_Model(10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer =torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
epochs = 10
losses = []
for epoch in range(epochs):
    for images,labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        y_pred = model(images)
        loss = loss_fn(y_pred,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)

        

100%|██████████| 1407/1407 [00:18<00:00, 74.79it/s] 


In [16]:
losses[len(losses)-1]

tensor(0.0050, device='cuda:0', grad_fn=<NllLossBackward0>)